# Init

In [1]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
from psycopg2.extras import NamedTupleCursor
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [2]:
from data_manager.database_manager import db_fetchone,db_no_fetch,db_fetchall
from core.utils.log import log_info,log_error
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        log_info('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchone().version
                log_info(f"PostgreSQL database version: {db_version}")
                log_info(f"PostgreSQL connection status: {conn.info.status}")
                log_info(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        log_error(error)
        conn = None


In [3]:
conn=init_connection(dsn)

2021-07-25 10:36:52.808 INFO    root: Connecting to the PostgreSQL database...
2021-07-25 10:36:52.839 INFO    root: PostgreSQL database version: PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit
2021-07-25 10:36:52.842 INFO    root: PostgreSQL connection status: 0
2021-07-25 10:36:52.844 INFO    root: You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'.


# 23/7/2021

In [17]:
from enum import Enum,IntEnum
from typing import NamedTuple

In [18]:
class Detection(NamedTuple):
    name: str
    prob: float

In [19]:
detection=Detection('car',0.95)

In [35]:
detection.name,detection.prob
detection

Detection(name='car', prob=0.95)

In [37]:
import numpy as np
x=np.array([1,2,3])

In [39]:
np.asarray(x)

array([1, 2, 3])

# 24/7/2021

In [13]:
query_project_SQL="""
                    SELECT
                        p.id,
                            p.name,
                            description,
                            deployment_id,
                            project_path,
                            dt.name as deployment_type
                    FROM
                        public.project p
                        LEFT JOIN deployment_type dt ON dt.id = p.deployment_id
                    WHERE
                        p.id = 2;"""

In [15]:
db_fetchone(query_project_SQL,conn)

Record(id=2, name='First Project', description='This project is for object detection', deployment_id=2, project_path='/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/project', deployment_type='Object Detection with Bounding Boxes')

### Get column names for DB

In [23]:

data_rows=[]
with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute("""select pt.id as "ID", pt.name as "Name",f.name as "Framework",model_path as "Model Path", pt.updated_at as "Date/Time" from pre_trained_models pt left join framework f on framework_id = f.id;""")
                column_names = [desc[0] for desc in cur.description]
                for row in cur:
                    data_rows.append(row)
print("Column names: {}\n".format(column_names))


Column names: ['ID', 'Name', 'Framework', 'Model Path', 'Date/Time']



In [14]:
from psycopg2.extras import NamedTupleCursor
with conn.cursor(cursor_factory=NamedTupleCursor) as cur:
    cur.execute("""
                select pt.id as "ID", pt.name as "Name",f.name as "Framework",model_path as "Model Path", pt.updated_at as "Date/Time" from pre_trained_models pt left join framework f on framework_id = f.id;
                    """)
    return_all=cur.fetchall()

In [111]:
SQL_msg="""
    select * from json_each_text('{"a":"foo", "b":"bar"}')"""

                    
return_all,column_names=db_fetchall(SQL_msg,conn=conn,fetch_col_name=True)

In [112]:
for i in return_all:
    display((i))

Record(key='a', value='foo')

Record(key='b', value='bar')

In [22]:
column_names=None

In [61]:
import pandas as pd

####  Create Pandas Dataframe using PostgreSQL column names

In [91]:
df=pd.DataFrame(return_all,columns=column_names)
display(df)

,ID,Name,Framework,Deployment Type,Model Path
0,1,[TF] SSD MobileNet V2 FPNLite 320x320,TensorFlow,Object Detection with Bounding Boxes,./pre-trained-models/ssd_mobilenet_v2_fpnlite_...
1,2,[TF] SSD ResNet50 V1 FPN 640x640 (RetinaNet50),TensorFlow,Object Detection with Bounding Boxes,./pre-trained-models/ssd_resnet50_v1_fpn_640x6...


In [92]:
from pathlib import Path
x=Path.home()
x

PosixPath('/home/rchuzh')

In [67]:
y=x/'./programming'

In [68]:
y

PosixPath('/programming')

In [94]:
DATA_DIR = Path.home() / '.local/share/integrated-vision-inspection-system/app_media'
DATA_DIR

PosixPath('/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media')

# 25/7/2021

In [63]:
from core.utils.helper import create_dataframe

In [90]:
df=create_dataframe(return_all,column_names=column_names)
df2=df.loc[(df["Framework"]=="TensorFlow"),"ID":"Framework"]
display(df2.iloc[10:20])

,ID,Name,Framework
No.,,,


In [57]:
x=pd.Series([True],index=["Framework"])
x

Framework    True
dtype: bool

In [97]:
name_list=[PT.Name for PT in return_all]
name_list

['My Second Dataset', 'My Fourth Dataset']

In [28]:
x=[1,2,3,4]
x.insert(0,"Select")
x

['Select', 1, 2, 3, 4]

In [98]:
for dataset in return_all:
    print(list(dataset))

[2, 'My Second Dataset', 4, datetime.datetime(2021, 7, 24, 14, 25, 45, 68588, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))]
[4, 'My Fourth Dataset', 4, datetime.datetime(2021, 7, 24, 14, 25, 45, 68588, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))]


In [113]:
import json

In [114]:
json.dumps(None)

'null'

In [115]:
from psycopg2.extras import Json

In [121]:
print(Json({}))

'{}'


In [127]:
class random:
    def __init__(self,name,age) -> None:
        self.name=name
        self.age=age

In [128]:
test=random("Alex",22)

In [130]:
def transformer(random:random):
    random.name="Mandy"
    

In [131]:
transformer(test)

In [132]:
test.name

'Mandy'